In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ast

# Load the dataset
file_path = 'TRADES_CopyTr_90D_ROI.csv'  # Use the correct file path in your system
df = pd.read_csv(file_path)

# Function to parse Trade_History and extract relevant details
def parse_trade_history(trade_history):
    parsed_trades = ast.literal_eval(trade_history)
    trade_details = []
    
    for trade in parsed_trades:
        trade_info = {
            'time': trade['time'],
            'symbol': trade['symbol'],
            'side': trade['side'],
            'positionSide': trade['positionSide'],
            'price': trade['price'],
            'quantity': trade['quantity'],
            'qty': trade['qty'],
            'realizedProfit': trade['realizedProfit']
        }
        trade_details.append(trade_info)
    
    return trade_details

# Handle missing values and parse Trade_History
df['Trade_History'] = df['Trade_History'].fillna('[]')
df['Parsed_Trades'] = df['Trade_History'].apply(parse_trade_history)

# Function to calculate metrics (ROI, PnL, Sharpe Ratio, etc.)
def calculate_metrics(trades):
    total_positions = len(trades)
    win_positions = sum(1 for trade in trades if trade['realizedProfit'] > 0)
    total_profit = sum(trade['realizedProfit'] for trade in trades)
    roi = total_profit / sum(trade['quantity'] for trade in trades) if sum(trade['quantity'] for trade in trades) > 0 else 0
    win_rate = win_positions / total_positions if total_positions > 0 else 0
    
    rois = [trade['realizedProfit'] / trade['quantity'] if trade['quantity'] > 0 else 0 for trade in trades]
    avg_roi = np.mean(rois) if rois else 0
    std_dev_roi = np.std(rois) if len(rois) > 1 else 0
    sharpe_ratio = avg_roi / std_dev_roi if std_dev_roi > 0 else 0
    
    running_total = 0
    peak = 0
    mdd = 0
    for trade in trades:
        running_total += trade['realizedProfit']
        peak = max(peak, running_total)
        mdd = max(mdd, peak - running_total)
    
    metrics = {
        'ROI': roi,
        'PnL': total_profit,
        'Sharpe_Ratio': sharpe_ratio,
        'MDD': mdd,
        'Win_Rate': win_rate,
        'Win_Positions': win_positions,
        'Total_Positions': total_positions
    }
    
    return metrics

# Apply the function to each account's parsed trades and create a new DataFrame for metrics
metrics_data = df['Parsed_Trades'].apply(calculate_metrics)
metrics_df = pd.DataFrame(metrics_data.tolist(), index=df['Port_IDs'])

# Assign weights to the metrics and calculate a score
weights = {
    'ROI': 0.3,
    'PnL': 0.3,
    'Sharpe_Ratio': 0.2,
    'MDD': 0.1,
    'Win_Rate': 0.05,
    'Win_Positions': 0.05
}

# Calculate weighted score
metrics_df['Score'] = (
    metrics_df['ROI'] * weights['ROI'] +
    metrics_df['PnL'] * weights['PnL'] +
    metrics_df['Sharpe_Ratio'] * weights['Sharpe_Ratio'] +
    metrics_df['MDD'] * weights['MDD'] +
    metrics_df['Win_Rate'] * weights['Win_Rate'] +
    metrics_df['Win_Positions'] * weights['Win_Positions']
)

# Rank accounts based on score
metrics_df['Rank'] = metrics_df['Score'].rank(ascending=False)

# Get the top 20 accounts
top_20_accounts = metrics_df.sort_values(by='Rank').head(20)

# Save the top 20 ranked accounts to a CSV file
top_20_accounts.to_csv('top_20_accounts.csv')

# Display the top 20 ranked accounts
print(top_20_accounts)


                          ROI           PnL  Sharpe_Ratio           MDD  \
Port_IDs                                                                  
4020204877254599680  0.003116  71998.855953     -0.088418  21338.051188   
3999240873283311617  0.005437  42574.473679      0.591594    300.813848   
4021669203289716224  0.004044  26427.331592      0.040751   2537.866629   
3960874214179953664  0.002333  19567.471286      0.116733   2234.000000   
3907081197088384000  0.010992  18015.997370      0.784622   2060.707122   
4028701921959171840  0.004065  17601.401398      0.675152    585.677600   
3956076827719377409  0.010322  16790.012238      0.680458    635.050015   
3986814617275053313  0.008716  16337.461881      1.288733      2.381947   
3788465932399412480  0.008368  13960.966457      0.217463   1772.219440   
4022565861939831809  0.002331  14197.577076      0.597433    385.075554   
3987739404272887297  0.005570  12464.606315      0.151787   3386.656094   
4008711265867865600  0.00